In [2]:
import xpress as xp

In [3]:
a_1 = xp.problem()

a_1.read("avec_contrainte_couplante\problem-1-1--optim-nb-1.mps")

Reading Problem Pb Solve
Problem Statistics
        1346 (      1 spare) rows
        2184 (      0 spare) structural columns
        4704 (   1848 spare) non-zero elements
Global Statistics
           0 entities        0 sets        0 set members


In [5]:
a_1.solve()

FICO Xpress v8.14.8, Hyper, solve started 9:53:22, Dec 5, 2023
Heap usage: 972KB (peak 972KB, 637KB system)
Minimizing LP Pb Solve using up to 8 threads and up to 31GB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
      1346 rows         2184 cols         4704 elements
Presolved problem has:
         0 rows            0 cols            0 elements
Presolve finished in 0 seconds
Heap usage: 1084KB (peak 2227KB, 637KB system)
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0       20065090.33      D      0     0        .000000     0
Uncrunching matrix
Optimal solution found
Dual solved problem
  0 simplex iterations in 0.00 seconds at time 0

Final objective                       : 2.006509032659758e+07
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max dual violation        (abs/rel) :       0.0 /       0.0
  Max complementarity viol. (abs/rel) :       0.0 /       0.0


In [6]:
print("solution of the original problem: ", a_1.getVariable(), "-->",
      a_1.getSolution())

solution of the original problem:  [NTCDirect::link<area$$fictive_watervalues_pump>::hour<0>                                                                        , NTCDirect::link<area$$fictive_watervalues_turb>::hour<0>                                                                        , DispatchableProduction::area<area>::ThermalCluster<Th1>::hour<0>                                                                , DispatchableProduction::area<area>::ThermalCluster<Th2>::hour<0>                                                                , DispatchableProduction::area<area>::ThermalCluster<Th3>::hour<0>                                                                , PositiveUnsuppliedEnergy::area<area>::hour<0>                                                                                   , NegativeUnsuppliedEnergy::area<area>::hour<0>                                                                                   , PositiveUnsuppliedEnergy::area<fictive_watervalues_pum

In [9]:
a_1.getrhs()

TypeError: function missing required argument 'rhs' (pos 1)

In [21]:
cst = a_1.getConstraint()
binding_id = [i for i in range(len(cst)) if "WeeklyWaterAmount" in cst[i].name]

In [24]:
a_1.chgrhs(binding_id,[100000])

In [25]:
a_1.solve()

FICO Xpress v8.14.8, Hyper, solve started 10:04:51, Dec 5, 2023
Heap usage: 3480KB (peak 4246KB, 637KB system)
Minimizing LP Pb Solve using up to 8 threads and up to 31GB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
      1346 rows         2184 cols         4704 elements
Presolved problem has:
       673 rows         1680 cols         3192 elements
Presolve finished in 0 seconds
Heap usage: 3756KB (peak 4246KB, 637KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e-05,  1.00e+05] / [ 1.00e-05,  1.00e+05]
  Objective      [min,max] : [ 5.01e-04,  3.00e+00] / [ 5.67e-04,  3.00e+00]
Autoscaling applied standard scaling

Starting parallel dual simplex, using up to 8 threads
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0       924845.6444      D      3     0        .000000     0
     5  